In [6]:
from openff.toolkit import Molecule, Topology, ForceField
from openff.interchange import Interchange

In [35]:
test_molecule = Molecule.from_smiles("Nc1ccccc1C(=O)O")
test_molecule.visualize()
test_molecule.generate_conformers(n_conformers=1)
print(test_molecule.conformers[0])

[[0.4421643070641558 2.1253203160654532 -0.12163827038694287] [-0.24566094209545059 0.8994669147996079 -0.06867502677082368] [-1.6376146577327826 0.8665166856614634 -0.1128520719952269] [-2.3160418305433113 -0.3373637631291373 -0.06095298134116517] [-1.565362009455285 -1.4937107763282507 0.035451745149244485] [-0.19073877511085624 -1.4643989186733517 0.07928024374861571] [0.48365439241849373 -0.2626259030702902 0.027379492734441478] [1.9479313427456613 -0.21804992643644686 0.07326377925053157] [2.5890824075851167 0.8388766079811009 0.02924292879140602] [2.651881630655574 -1.4209806408775538 0.17097782039919449] [0.29546122951830983 2.8201089753044166 0.6382108578341174] [1.1004652813023843 2.3962574653381448 -0.8786059795929514] [-2.179291340832703 1.8147804886161323 -0.18946541078253418] [-3.400333657667819 -0.34579688250831353 -0.09642665582090501] [-2.061251997148322 -2.449227790126704 0.07802987789585838] [0.4249159076256674 -2.3591040352536514 0.1550361988137158] [3.66073871167109

In [20]:
topology = Topology.from_molecules(test_molecule)
sage = ForceField("openff-2.0.0.offxml")
interchange = Interchange.from_smirnoff(topology=[test_molecule], force_field=sage)
interchange.minimize()

In [40]:
import openmm
import openmm.unit
interchange.box = [[2,0,0],[0,2,0],[0,0,2]]*openmm.unit.nanometer

integrator = openmm.LangevinMiddleIntegrator(
    293.15 * openmm.unit.kelvin,
    1.0 / openmm.unit.picosecond,
    2.0 * openmm.unit.femtosecond,
)
barostat = openmm.MonteCarloBarostat(
    1.00 * openmm.unit.bar,
    293.15 * openmm.unit.kelvin,
    25,
)
simulation = interchange.to_openmm_simulation(
    integrator=integrator,
    additional_forces=[barostat]
)

interchange.to_top("test.top")

simulation.context.setVelocitiesToTemperature(300*openmm.unit.kelvin)
dcd_reporter = openmm.app.DCDReporter("test.dcd", 1000)
state_data_reporter = openmm.app.StateDataReporter(
    "test.log",
    1000,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(state_data_reporter)

simulation.step(10000)

[[2 0 0] [0 2 0] [0 0 2]] nanometer
